<DIV ALIGN=CENTER>

# Introduction to Social Media: Web
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore 

Use BeautifulSoup to process web page.

Language classification, using Wikipedia?

Process UIUC Course explorer?

Newspaper?

-----


### Structured Text Parsing

To parse structured text, like an XML or an HTML document, we can use
the Python [Beautiful Soup][bs] library. This library uses an XML/HTML
parser to build a DOM tree, and Beautiful Soup then provides traversal
methods to access and modify the DOM for a specific document. Beautiful
Soup has been extremely popular for the ease with which it allows web
scraping, for example, you can pull data out of an HTML table. But it is
more powerful than this, as it allows you to easily parse and manipulate
any XML document.

To use Beautiful Soup, we first need to import the library, and then
create a BeautifulSoup object that provides access to the parsed data.
Document elements, like `body` or `table` are directly accessed from the
parsed tree; and element attributes or data can be easily extracted,
deleted, or replaced. If required, new data can also be added to an
existing document, allowing for the dynamic creation of a new document.
These capabilities are demonstrated in the following code cells.

-----
[bs]: http://www.crummy.com/software/BeautifulSoup/




In [ ]:
# Lets parse our HTML document

# We use BeautofulSoup version 4
from bs4 import BeautifulSoup
  
soup = BeautifulSoup(html)

# Now lets print out the start of the HTMl file
print(soup.prettify()[:134])

In [ ]:
# We can access document elements directly
print('code element:= ', soup.pre)
print('value:', soup.pre.string)

# We can access parent data
print('parent element: ', soup.pre.parent.name)

In [ ]:
# We can directly access elemnt attributes

print('font class attribute: ', soup.font['size'])

In [ ]:
# We can access an entire element's content
print(soup.b)

In [ ]:
# We can find all occurances of a particular element

for el in soup.find_all('p'):
    print(el)

In [ ]:
# We can also change data in the document

soup.body['class'] = 'newClass'

print("\nBody class attribute = ", soup.body['class'])



In [ ]:
# We can delete elements

myCode = soup.pre.extract()

print(soup.pre)

In [ ]:
# We can select elements based on CSS Selectors
target = soup.select('font[size]')
print(target)

In [ ]:
# We need to pull out the first element in the list to get tag
# Now we can insert our table back into the DOM

target[0].insert_after(myCode)
print(soup.pre)

In [ ]:
# We can also insert entirely new elements.

# First we create a new element (tag)
tag = soup.new_tag('h3', id='h3id')
tag.string = 'A New Header'

# Now we can append (in this case we put the new element at the end of the body)

soup.body.append(tag)

# Show the result
print(soup.h3)

-----

While Beautiful Soup provides a great deal of power and simplicity in
DOM parsing and element retrieval, the full power of parsing a document
requires the use of regular expressions. Given a regular expression, the
first task in Python is to compile the RE, which is done by using the
`compile` method in the `re` module. This is demonstrated in the
following code cell where we use a regular expression to find the
element containing `Docker` to display HTML entities that contain the
word _Docker_.

-----



In [ ]:
# We need the re module
import re 

# Open and parse our XML document
soup = BeautifulSoup(html)

# Findelements containing the Docker string
for el in soup.find_all(text=re.compile('Docker')):
    print(el.parent)

-----

## Student Activity

In the preceding cells, we used . Now that you
have run the Notebook, go back and make the following changes to see how
the results change.

1. Change the ?
2. Change the ?
3. Try making a ?

Finally, try applying a (discuss in class forums?

you should work to improve your Python text data
processing skills. Specific problems you can attempt include the
following:

1. Modify the first String Processing code to convert all text to
lowercase characters before accumulating the word counts.

2. Use the Python set to obtain the list of unique words in the text
message.

3. Use Regular Expressions to remove the email encoding characters from
the message text.

Additional, more advanced problems:

1. Save several emails from within your mail reader and modify the
Python code to process them in bulk to extract out the sender, date
sent, and subject.

2. Save several webpages (perhaps by using wget), and modify the
BeautifulSoup code example to parse out and display the page title, any
Javascript code libraries, and any css style file references.

-----
